# Deriving some equations

The last round key is noted $K$:

$$
    K =
    \begin{pmatrix}
        K_{1} & K_{5} & K_{ 9} & K_{13}\\
        K_{2} & K_{6} & K_{10} & K_{14}\\
        K_{3} & K_{7} & K_{11} & K_{15}\\
        K_{4} & K_{8} & K_{12} & K_{16}
    \end{pmatrix}
$$

The cipher text is noted $Y$:

$$
    Y = 
    \begin{pmatrix}
        Y_{1} & Y_{5} & Y_{ 9} & Y_{13}\\
        Y_{2} & Y_{6} & Y_{10} & Y_{14}\\
        Y_{3} & Y_{7} & Y_{11} & Y_{15}\\
        Y_{4} & Y_{8} & Y_{12} & Y_{16}
    \end{pmatrix}
$$

Reversing the final state $Y$ to 9-th round output $Y^9$ gives:
$$
    Y^9 = 
    \begin{pmatrix}
        \operatorname{S^{-1}}(K_{ 1} + Y_{ 1}) &
        \operatorname{S^{-1}}(K_{ 5} + Y_{ 5}) &
        \operatorname{S^{-1}}(K_{ 9} + Y_{ 9}) &
        \operatorname{S^{-1}}(K_{13} + Y_{13})\\

        \operatorname{S^{-1}}(K_{14} + Y_{14}) &
        \operatorname{S^{-1}}(K_{ 2} + Y_{ 2}) &
        \operatorname{S^{-1}}(K_{ 6} + Y_{ 6}) &
        \operatorname{S^{-1}}(K_{10} + Y_{10})\\

        \operatorname{S^{-1}}(K_{11} + Y_{11}) &
        \operatorname{S^{-1}}(K_{15} + Y_{15}) &
        \operatorname{S^{-1}}(K_{ 3} + Y_{ 3}) &
        \operatorname{S^{-1}}(K_{ 7} + Y_{ 7})\\

        \operatorname{S^{-1}}(K_{ 8} + Y_{ 8}) &
        \operatorname{S^{-1}}(K_{12} + Y_{12}) &
        \operatorname{S^{-1}}(K_{16} + Y_{16}) &
        \operatorname{S^{-1}}(K_{ 4} + Y_{ 4})
    \end{pmatrix}
$$

The fault is injected between 7-th round `MixColumns` and 8-th round `MixColumns`.  
After the following `MixColumns` (8-th round's one), the differential state consist of a single column $(A, B, C, D)$.  
Depending on the location of that column (which depends back on the position the fault was injected at), the differential state obtained after the next `MixColumn` is described below.

$$
    \begin{pmatrix}
        A & 0 & 0 & 0\\
        B & 0 & 0 & 0\\
        C & 0 & 0 & 0\\
        D & 0 & 0 & 0
    \end{pmatrix}
    \longrightarrow
    \begin{pmatrix}
        2A &  D &  C & 3B\\
        A &  D & 3C & 2B\\
        A & 3D & 2C &  B\\
        3A & 2D &  C &  B
    \end{pmatrix}
$$

$$
    \begin{pmatrix}
        0 & A & 0 & 0\\
        0 & B & 0 & 0\\
        0 & C & 0 & 0\\
        0 & D & 0 & 0
    \end{pmatrix}
    \longrightarrow
    \begin{pmatrix}
        3B & 2A &  D &  C\\
        2B &  A &  D & 3C\\
         B &  A & 3D & 2C\\
         B & 3A & 2D &  C
    \end{pmatrix}
$$

$$
    \begin{pmatrix}
        0 & 0 & A & 0\\
        0 & 0 & B & 0\\
        0 & 0 & C & 0\\
        0 & 0 & D & 0
    \end{pmatrix}
    \longrightarrow
    \begin{pmatrix}
         C & 3B & 2A &  D\\
        3C & 2B &  A &  D\\
        2C &  B &  A & 3D\\
         C &  B & 3A & 2D
    \end{pmatrix}
$$

$$
    \begin{pmatrix}
        0 & 0 & 0 & A\\
        0 & 0 & 0 & B\\
        0 & 0 & 0 & C\\
        0 & 0 & 0 & D
    \end{pmatrix}
    \longrightarrow
    \begin{pmatrix}
     D &  C & 3B & 2A\\
     D & 3C & 2B &  A\\
    3D & 2C &  B &  A\\
    2D &  C &  B & 3A
    \end{pmatrix}
$$

Equating the adequate differential state (from above ones) to $Y^9 \oplus Y'^9$ gives the equations used for the first stage reduction.

In [1]:
from IPython.core.display import display
import sympy as sp

In [4]:
S = sp.Function('S')
invS = sp.Function('S^{-1}')

P1, P2, P3, P9, P11, P13, P14 = 1, *sp.symbols('P2 P3 P9 P11 P13 P14')

def SubBytes(X):
    return sp.Matrix([
        [S(X[0, 0]), S(X[0, 1]), S(X[0, 2]), S(X[0, 3])],
        [S(X[1, 0]), S(X[1, 1]), S(X[1, 2]), S(X[1, 3])],
        [S(X[2, 0]), S(X[2, 1]), S(X[2, 2]), S(X[2, 3])],
        [S(X[3, 0]), S(X[3, 1]), S(X[3, 2]), S(X[3, 3])]
    ])

def invSubBytes(X):
    return sp.Matrix([
        [invS(X[0, 0]), invS(X[0, 1]), invS(X[0, 2]), invS(X[0, 3])],
        [invS(X[1, 0]), invS(X[1, 1]), invS(X[1, 2]), invS(X[1, 3])],
        [invS(X[2, 0]), invS(X[2, 1]), invS(X[2, 2]), invS(X[2, 3])],
        [invS(X[3, 0]), invS(X[3, 1]), invS(X[3, 2]), invS(X[3, 3])]
    ])

def ShiftRows(X):
    return sp.Matrix([
        [X[0, 0], X[0, 1], X[0, 2], X[0, 3]],
        [X[1, 1], X[1, 2], X[1, 3], X[1, 0]],
        [X[2, 2], X[2, 3], X[2, 0], X[2, 1]],
        [X[3, 3], X[3, 0], X[3, 1], X[3, 2]]
    ])

def invShiftRows(X):
    return sp.Matrix([
        [X[0, 0], X[0, 1], X[0, 2], X[0, 3]],
        [X[1, 3], X[1, 0], X[1, 1], X[1, 2]],
        [X[2, 2], X[2, 3], X[2, 0], X[2, 1]],
        [X[3, 1], X[3, 2], X[3, 3], X[3, 0]]
    ])

def MixColumns(X):
    M = sp.Matrix([
        [P2, P3, P1, P1],
        [P1, P2, P3, P1],
        [P1, P1, P2, P3],
        [P3, P1, P1, P2]
    ])
    return M * X

def invMixColumns(X):
    M = sp.Matrix([
        [P14, P11, P13,  P9],
        [ P9, P14, P11, P13],
        [P13,  P9, P14, P11],
        [P11, P13,  P9, P14]
    ])
    return M * X

In [5]:
ind = [[1, 5,  9, 13],
       [2, 6, 10, 14],
       [3, 7, 11, 15],
       [4, 8, 12, 16]]

K = sp.symbols('K0:17')
K = sp.Matrix([[K[ind[i][j]] for j in range(4)] for i in range(4)])

Y = sp.symbols('Y0:17')
Y = sp.Matrix([[Y[ind[i][j]] for j in range(4)] for i in range(4)])

A, B, C, D = sp.symbols('A B C D')

Delta1 = sp.Matrix([
       [A, 0, 0, 0],
       [B, 0, 0, 0],
       [C, 0, 0, 0],
       [D, 0, 0, 0]
])

Delta2 = sp.Matrix([
       [0, A, 0, 0],
       [0, B, 0, 0],
       [0, C, 0, 0],
       [0, D, 0, 0]
])

Delta3 = sp.Matrix([
       [0, 0, A, 0],
       [0, 0, B, 0],
       [0, 0, C, 0],
       [0, 0, D, 0]
])

Delta4 = sp.Matrix([
       [0, 0, 0, A],
       [0, 0, 0, B],
       [0, 0, 0, C],
       [0, 0, 0, D]
])

display(invSubBytes(invShiftRows(Y+K)))

display(MixColumns(ShiftRows(Delta1)))
display(MixColumns(ShiftRows(Delta2)))
display(MixColumns(ShiftRows(Delta3)))
display(MixColumns(ShiftRows(Delta4)))

Matrix([
[  S^{-1}(K1 + Y1),   S^{-1}(K5 + Y5),   S^{-1}(K9 + Y9), S^{-1}(K13 + Y13)],
[S^{-1}(K14 + Y14),   S^{-1}(K2 + Y2),   S^{-1}(K6 + Y6), S^{-1}(K10 + Y10)],
[S^{-1}(K11 + Y11), S^{-1}(K15 + Y15),   S^{-1}(K3 + Y3),   S^{-1}(K7 + Y7)],
[  S^{-1}(K8 + Y8), S^{-1}(K12 + Y12), S^{-1}(K16 + Y16),   S^{-1}(K4 + Y4)]])

Matrix([
[A*P2,    D,    C, B*P3],
[   A,    D, C*P3, B*P2],
[   A, D*P3, C*P2,    B],
[A*P3, D*P2,    C,    B]])

Matrix([
[B*P3, A*P2,    D,    C],
[B*P2,    A,    D, C*P3],
[   B,    A, D*P3, C*P2],
[   B, A*P3, D*P2,    C]])

Matrix([
[   C, B*P3, A*P2,    D],
[C*P3, B*P2,    A,    D],
[C*P2,    B,    A, D*P3],
[   C,    B, A*P3, D*P2]])

Matrix([
[   D,    C, B*P3, A*P2],
[   D, C*P3, B*P2,    A],
[D*P3, C*P2,    B,    A],
[D*P2,    C,    B, A*P3]])